# Managing Models at Scale Using a Model Registry

#### As you begin to deploy multiple models and manage multiple model versions, ensuring core architectural practices such as governance, traceability, and recoverability are followed is challenging without using a model registry. A model registry is a central store containing metadata specific to a model version. It includes information on how the model was built, the performance of that model, as well as where and how the model is deployed. Model registry services or solutions often include additional capabilities, such as approval workflows and notifications.

## Using a model registry

#### Regardless of the implementation, the key metadata to consider includes model version identifiers, and the following information about each model version registered:

- Model inputs: These include metadata related to the inputs and versions of those inputs used to train the model. This can include inputs such as the name of the Amazon S3 bucket storing the training data, training hyperparameters, and the Amazon Elastic Container Registry (ECR) repository or container image used for training.
- Model performance: This includes model evaluation data such as training and validation metrics.
- Model artifact: This includes metadata about the training model artifact. At a minimum, this includes the name of the Amazon S3 bucket storing the model artifact, as well as the name of the object (for example, model.tar.gz).
- Model deployment: This includes metadata relating to the deployment of a model. This includes information such as the environment(s) a model version is deployed to, or the inference code that can be used for the registered model.

#### Similar considerations exist for tracking and storing model deployment data. The metadata tracked for model deployments should provide enough information to package the model for deployment using Amazon SageMaker, to a real-time endpoint, or using batch transform. This should also allow someone to easily identify where a given model version is deployed, as well as how it is packaged for deployment and consumption. Figure 8.2 illustrates an example of the inputs, deployment stages, and artifacts to consider for tracking across the SageMaker options for deploying models:

#### If you had a couple of models to manage, you could potentially track the previous information using a simple method, such as a spreadsheet. However, as you begin to scale to 20, 100, or thousands of models, that mechanism for tracking model metadata no longer scales. Centrally storing and tracking the information (shown in Figures 8.1 and 8.2) for each model version provides the following benefits:

- Operational efficiencies: A model registry provides tracking and visibility into key inputs used to build a specific model version, output artifacts, and information about the deployment stages aligned to that version. Having this metadata allows for the ability to quickly understand how a model was built, how the model performed, information about the trained model artifact, and also provides the ability to track the environment(s) a specific version is deployed to.

- Recoverability: To be able to recover a deployed model or roll back to a previous version, you need to have visibility to the inputs and input versions used to create a deployable artifact or a deployed model. In the event of system or human error, you can recover to a specific point in time using the metadata stored in the model registry, combined with protected versioned inputs. As an example, if an administrator were to accidentally delete a model endpoint, it should be easy to identify the artifacts needed to recreate that endpoint. This can be identified using metadata stored in the model registry that points to the location of the versioned model artifact, in combination with the versioned inference container image.

- Pipeline sources and triggers: Often there is a need to bridge the model build and model deployment environments. This is typical in large enterprises that have central deployment teams, or in organizations that separate model build and model deployment roles. A model registry provides a mechanism to capture the minimum metadata needed for visibility into how a model is built. However, it can also be used to trigger approval workflows and downstream deployments.

## Amazon SageMaker model registry

#### The Amazon SageMaker model registry is a managed service that allows you to centrally catalog models, manage model versions, associate metadata with your model versions, and manage the approval status of a model version. The service is continuously evolving with new features, so the information contained in this section is current as of the publication date. It's always recommended to validate the current features and capabilities with the official documentation for the Amazon SageMaker model registry (https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry.html). The SageMaker model registry is optimized for use in conjunction with Amazon SageMaker Pipelines and projects; however, it can also be used independently as well.


## The main components of the SageMaker model registry include the following:

- Model registry: This is the central store containing model groups and it exists at the AWS account and AWS region levels. Cross-account privileges can be set up to interact with the model registry from other AWS accounts.
- Model groups: Model groups are a logical grouping. They allow you to track different model versions that are related to, or grouped by, the same machine learning problem.
- Model packages: Model packages are registered models or specific versions of a model.

#### It's possible to register a model as either versioned or unversioned. Model packages that are versioned are part of a model group, and unversioned model packages are not part of a model group. The benefit of using a model group, or a versioned model, is the ability to logically group and manage models that are related, as well as provide the ability to automatically version models related to a specific machine learning (ML) problem. It's recommended to register your models using model groups with registered models that are versioned. This is the default setting.

- Inference specification: A series of parameters that provide detailed information and guidance on hosting the model for inference. Information passed includes data such as the Amazon ECR data. This contains the inference code image, the Amazon S3 bucket containing the trained model artifact, and the supported instance types when hosting the model for either real-time inference or for batch inference. For example, if a model requires GPU for inference, that can be captured in the registry.

- Model metrics: Model evaluation metrics across evaluated categories, such as statistical bias in a model, or model quality.

- Validation specification: Information about the SageMaker batch transform job(s) that were used to validate the model package (if applicable).

- Algorithm specification: Details about the algorithm(s) used to create the model, as well as the Amazon S3 bucket containing the trained model artifact.

- Metadata properties: These properties contain metadata for the CodeCommit commit ID, author of the source, the SageMaker Pipelines project ID, and the name of the CodeCommit repository. While they are not restricted for use outside Amazon SageMaker Pipelines, they are direct pointers to SageMaker Pipelines project resources.

- Model approval status: This parameter is used to indicate whether a model is approved for deployment. This parameter can be used to manage workflows. In the case of SageMaker Pipelines projects, the automated workflow triggers are automatically set up based on the status of this field. If a model status is changed to approved, a downstream deployment workflow can be triggered.

#### Amazon SageMaker's model registry is fully managed, meaning there are no servers to manage. It also natively integrates into SageMaker Pipelines, providing the ability to integrate directly with the model registry as a native step in your model build pipeline. It does this using the RegisterModel step.

#### For example, if you build a model build pipeline that contains the automated steps for data processing, training, and model evaluation, you can add a conditional step to validate the evaluation metric. If the evaluation metric is above a specified threshold (for example, accuracy > 90%), the pipeline can then be configured to automatically register your model.

#### SageMaker's model registry also integrates natively with SageMaker Pipelines projects. Projects allow you to automatically provision MLOps pipelines and provision patterns that take advantage of the model registry. SageMaker projects can be used to automatically set up the model package group, as well as the approval workflows that can be used to trigger the pre-configured downstream deployment pipeline.

## Building a custom model registry

#### A model registry can also be built using AWS services. Building a custom registry requires more effort to build the solution, set up the integrations between AWS services, set up the ML pipeline integrations, and then manage the solution. However, a custom registry also offers the ability to completely customize a registry to meet the needs specific to your use case. This could include requirements specific to tracking more granular metadata, or requirements to support multiple ML services/platforms. In this section, we'll review one pattern for creating a custom model registry using AWS services.

## Creating a model package group

#### A model package group contains a collection of model packages or model versions. A model package group is not required for registering a model package; however, it is recommended for the manageability of your model versions across ML use cases. A model package group can contain one or more model packages.

#### Creating a model package group involves a method that accepts only a few parameters on input to configure, as follows:

In [ ]:
import time
model_package_group_name = "air-quality-" + str(round(time.time()))
model_package_group_input_dict = {
"ModelPackageGroupName" : model_package_group_name,
"ModelPackageGroupDescription" : "model package group for air quality models",
"Tags": [
            {
            "Key": "MLProject",
             "Value": "weather"
              }
]  
}
create_model_pacakge_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
print('ModelPackageGroup Arn : {}'.format(create_model_pacakge_group_response['ModelPackageGroupArn']))

#### The preceding code is used to create a model package group that can then be used by ML builders, as well as with ML pipelines to register model packages (versions) for deployment. Configuration for a model package group requires only a model package group name and optionally a description and any tags you want to associate with the model group.

 - Establishing naming standards for model package groups: As the number of model package groups grows, having clear naming standards can help with easily identifying and searching for related model package groups. Some considerations may include a team identifier and/or project identifier. Because it's common to have more than one team working on models, a team identifier can help easily sort and search for models specific to a given team. It's also common to have more than one model used in an overall solution. In this case, it is valuable to have a way to group models related to a specific project or solution. This can be done through established naming conventions, as well as tagging.

- Utilizing tags for fine-grained access: In the preceding example, a tag of MLProject is created with the value of weather. In this case, let's assume a weather team is responsible for building weather-related models and only team members belonging to the weather team should be able to view model package groups with this tag. Resource tags can be used to establish conditional policies for access.

## Creating a model package

#### A model package is a model version that can exist outside of a model package group, referred to as unversioned, or inside a model package group, referred to as versioned. A model package outside of a model package group is referred to as unversioned because it's not using the versioning capabilities of a model package group. It's recommended to register model packages using model package groups for automatic management of model versions, and for added manageability as the number of model versions increases.

#### Amazon SageMaker has two concepts called model package. The two are independent of each other. The first example is a model package that is created to package a model for deployment using the CreateModel API. This is required to deploy your model using Amazon SageMaker and is discussed in the Amazon SageMaker documentation (https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-mkt-model-pkg-model.html). The second example, and the one we refer to in this chapter, is a model package specifically for Amazon SageMaker's model registry that is created using the CreateModelPackage API.

#### The CreateModelPackage API accepts several parameters on input. The high-level parameter categories were already covered in the section titled Amazon SageMaker model registry, so in this section, we'll include an example that uses those parameters to then register a model using our sample use case. In Chapter 12, Machine Learning Automated Workflows, we'll again discuss the model registry in the context of an ML pipeline, to demonstrate how a model registry can be integrated into your automated workflows. For now, we'll focus on registering a model package as an indication that it has passed initial model validation outside of a pipeline workflow.

#### In this case, the model has been trained and we've evaluated the training metrics. Once our model reaches the minimum threshold identified for our evaluation metric, we are ready to register the model package. Using the AWS Python SDK (boto3), we'll register the model package, as shown in the following code:

In [ ]:
modelpackage_inference_specification =  {
            "InferenceSpecification": {
            "Containers": [
            {
             "Image": xgboost_container,
             "ModelDataUrl": model_url
             }
             ],
             "SupportedContentTypes": [ "text/csv" ],
              "SupportedResponseMIMETypes": [ "text/csv" ],
  }
}

create_model_package_input_dict = {
            "ModelPackageGroupName" : model_package_group_name,
           "ModelPackageDescription" : "Model to predict air quality ratings using XGBoost",
            "ModelApprovalStatus" : "PendingManualApproval"
}

create_model_package_input_dict.update(modelpackage_inference_specification)
create_mode_package_response = sm_client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_mode_package_response["ModelPackageArn"]
print('ModelPackage Version ARN : {}'.format(model_package_arn))

#### ModelPackageGroupName is required to associate the model package with a model package group. This allows you to take advantage of automatic versioning, as previously discussed.

#### The model packages can then be viewed using the list_model_packages method, as well as within Amazon SageMaker Studio. To list the model package, use the following code:
- Creating versioned packages: Associate model packages with a model group by specifying the model package group when you create your model package. This allows for automatic versioning and grouping of use cases for easier management.

- Using model approval status: The optimal use of the model approval status field is to allow for peer reviews and trigger downstream deployment workflows using Amazon SageMaker projects. However, even without the use of Amazon SageMaker projects, the same field can be used to ensure data used to register a model passes a minimum set of criteria. For example, if there is a team standard to include explainability metrics for a registered model, then that ApprovalStatus can optionally be used after a peer review of the registered model to indicate minimum standards or criteria have been met for that model.

- Protecting the inputs/artifacts referred to in the model registry: Details contained in the model registry can be used to recreate or roll back deployed models; however, those resources need to be protected from unauthorized access or accidental deletion. For example, if an administrator accidentally deletes a SageMaker endpoint, it can still be easily recreated using the resources identified in the model registry. This would include the S3 object containing the model artifact, the S3 object with inference code (optional), and the ECR inference image. If any of those inputs are not available or cannot be guaranteed, then re-creating that endpoint may not be possible. Therefore, the metadata gives the information required, but there are still additional steps needed to protect inputs and artifacts.

- Considering tags when additional metadata is needed: The metadata within SageMaker's model registry is fixed to the input parameters that are defined in the API. However, tags can be used to supplement additional metadata. An example of the recommended use of tags here would be to capture the S3 version for resources such as the model artifact, in order to include more granularity on artifact tracking.

- Utilizing tags for fine-grained access: In the preceding example, a tag of MLProject is created with the value of weather. In this case, let's assume a weather team is responsible for building weather-related models and only team members from this team should be able to register new models to this model package group or other model package groups created with this tag. Resource tags can be used to establish conditional policies for access, in order to create model packages within specific model package groups. Resource tags can be used to establish conditional policies for access.

In [ ]:
import time
import os
from sagemaker import get_execution_role, session, image_uris
import boto3

region = boto3.Session().region_name

role = get_execution_role()

sm_client = boto3.client('sagemaker', region_name=region)

## 1. Create a Model Group

In [ ]:
import time
model_package_group_name = "air-quality-" + str(round(time.time()))
model_package_group_input_dict = {
 "ModelPackageGroupName" : model_package_group_name,
 "ModelPackageGroupDescription" : "model package group for air quality models",
 "Tags": [
     {
         "Key": "MLProject",
         "Value": "weather"
     }
 ]   
}
create_model_pacakge_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
print('ModelPackageGroup Arn : {}'.format(create_model_pacakge_group_response['ModelPackageGroupArn']))

## 2. Register a Model Version

#### First, we need to find the model_url that will be used as input to register the model version. Typically this is included as part of a pipeline; however, in this case we are registering the model outside of a pipeline so we need to pull the data from our previous training job that resulted in a candidate model that is performing well according to our objective metric.

In [ ]:
#Example: 
#training_job = 'sagemaker-xgboost-2021-07-28-02-43-50-684'
training_job = 'REPLACE WITH NAME OF TRAINING JOB'

training_job_response = sm_client.describe_training_job(
    TrainingJobName=training_job
)

model_url=training_job_response['ModelArtifacts']['S3ModelArtifacts']
print('Model Data URL', model_url)

# this line automatically looks for the XGBoost image URI and builds an XGBoost container.
# specify the repo_version depending on your preference.
xgboost_container = image_uris.retrieve(region=boto3.Session().region_name,
                          framework='xgboost', 
                          version='1.2-1')

print('XGBoost Container for Inference:', xgboost_container)

modelpackage_inference_specification =  {
    "InferenceSpecification": {
      "Containers": [
         {
            "Image": xgboost_container,
             "ModelDataUrl": model_url
         }
      ],
      "SupportedContentTypes": [ "text/csv" ],
      "SupportedResponseMIMETypes": [ "text/csv" ],
   }
 }

create_model_package_input_dict = {
    "ModelPackageGroupName" : model_package_group_name,
    "ModelPackageDescription" : "Model to predict air quality ratings using XGBoost",
    "ModelApprovalStatus" : "PendingManualApproval"  
}
create_model_package_input_dict.update(modelpackage_inference_specification)

create_model_package_response = sm_client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_mode_package_response["ModelPackageArn"]
print('ModelPackage Version ARN : {}'.format(model_package_arn))

## Let's view the detailed of our registered model

In [ ]:
sm_client.list_model_packages(ModelPackageGroupName=model_package_group_name)